In [105]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
%matplotlib inline

In [106]:
# neural network class defenition
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes for each layer - input, hidden, output
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # learning rate
        self.lr = learningrate
        
        # link weight matrices wih and who
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5),
        (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5),
        (self.onodes, self.hnodes))
        
        # activation function
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
    # train the network
    def train(self, inputs_list, targets_list):
        # convert inputs list in to 2D array
        inputs = np.array(inputs_list, ndmin = 2).T
        targets = np.array(targets_list, ndmin = 2).T
        
        # calculate signals coming into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals coming into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # calculate error (target - actual)
        output_errors = targets - final_outputs
        
        # calculate hidden layer errors
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # update weights for the links btwn hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs *
        (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # update weights for the links btwn input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs *
        (1.0 - hidden_outputs)), np.transpose(inputs))
        
        pass
    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list in to 2D array
        inputs = np.array(inputs_list, ndmin = 2).T
        
        # calculate signals coming into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        # calculate signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals coming into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        # calculate signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    

In [107]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

# learninig rate
learning_rate = 0.3

# create instance of neural network
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [108]:
# Load MNIST training data file into a list
#training_data_file = open("mnist_dataset/mnist_train_100.csv", 'r')
training_data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [109]:
# Train the neural network

# go through all records in the training data set
for record in training_data_list:
    # split the record by ',' commas
    all_values = record.split(',')
    # prepare data - scale and shift
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99 ) + 0.01
    # create target matrix
    targets = np.zeros(output_nodes) + 0.01
    # all_values[0] - is a Label of currect record
    targets[int(all_values[0])] = 0.99
    # train the network
    n.train(inputs, targets)
    pass
    

In [113]:
# Test the network
# load MNIST test data set file
#test_data_file = open("mnist_dataset/mnist_test_10.csv", 'r')
test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

# Scores for the network
scorecard = []

for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
#    print(correct_label, 'correct label')
    # prepare input data
    inputs = (np.asfarray(all_values[1:])/255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # index of the highest value in output array
    label = np.argmax(outputs)
#    print(label, "network`s answer")
#    print('------------------')
    # appent correct to the score list
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
pass

#print(scorecard)
# calculate network performance
scorecard_array = np.asarray(scorecard)
print('performance = ', scorecard_array.sum()/scorecard_array.size)
    

performance =  0.9411
